<a href="https://colab.research.google.com/github/EduGame3/Experiment/blob/master/PortOpcionesTDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 487 (delta 133), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (487/487), 16.25 MiB | 18.28 MiB/s, done.
Resolving deltas: 100% (316/316), done.


In [0]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)

#install.packages("Deriv") #paquete para derivar funciones
#library(Deriv)

library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12-0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 540698 bytes (528 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [0]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb<-function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#Definición de los parámetros a valorar de los n contratos del portafolio.


In [0]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.95 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia

btasadesc_oir="RiesgosFinancieros/2020-1/Insumos/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2020-1/Insumos/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2020-1/Insumos/tvoltiie_opc.txt"

####### INGRESE LOS DETALLES DE LOS ACTIVOS A VALORAR  ####### 
plazos_oir=cbind( 39,	115,	153,	245,	360) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0,0,0,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.07, 0.085, 0.078, 0.085, 0.08)
contratos_oir=cbind(2200, -2900, 2900, -460, 5000)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa, 0 si es simple
rendimientos=1 #1 si son por logaritmo, 0 si es por cocientes

#Carga de datos de los factores de riesgo para las opciones.
1- Carga de rho\
2- Tasa spot\
3- Volatilidad de tasa spot

In [0]:
%%R
#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)   #se quita la primera columna que corresponde a las fechas de las tasas.
  #(al final se agrega una columna con las fechas de las tasas escritas correctamente)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir]) #plazos de las tasas, son 16(1d,1W, 1M, 2M, 3M, 6M, 9M, 1Y,...)
  #la primera columna de x1_orig_oir corresponde al valor de la tasa para el plazo nodos1_oir[1] es decir 1d y así para cada columna.

  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])

  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])


#Integración de Insumos.

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. \
Posteriormente, filtramos cada histórico de  factor de riesgo conseguimos la intersección de todos.

In [0]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(x1_orig_oir[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #guber, st (bonde), spot, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #guber, st (bonde),spot,opciones ir spot, opc ir vp, opc ir vol

n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Histórico de todas las tasas intersectadas.

#historia de los factores de riesgo (tasas) para las opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)


# Pasos para medir el riesgo por el método de simulación Monte Carlo.

Para todos los instrumentos hay que definir y calcular los siguientes elementos:
1. Historico de factores de riesgo
2. Vector de precios actual

In [0]:
%%R
#Interpolación de tasas y sus volatilidades; valuación de los instrumentos.

m=ncol(plazos_oir)  #número de instrumentos
x1=matrix(0,n,m)  
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

x01=x1[1,] #tasas de descuento (TIIE)   #****************************** por qué quedarse con la primera columna?
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
X_oir=cbind(x1,x2,x3) #matriz con los factores de riesgo por cada instrumento.


Integración de factores y cálculo de riesgo en conjunto.

In [0]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if=matrix(0,1,1)
n_if[1]=ncol(X_oir) #número de factores de riesgo 

#valor del portafolio
V0_port <- cbind(V0_oir)
V0T_port <- sum(V0_port)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(X_oir) #Factores de riesgo del portafolios de 1 instrumento financiero (muchas opciones)
#Cálculo de variaciones de los factores de riesgo: Delta_X DEL PORTAFOLIOS

if(rendimientos==1){
    DeltaX_port=as.matrix(log(X_port[1:(n-1),]/X_port[2:(n),]))
}else{
    DeltaX_port=as.matrix(X_port[1:(n-1)]/X_port[2:(n)]-1)
}
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

#print(V0T_port) #Suma de los valores de la valoración de los instrumentos.


#Simulación de factores de riesgo.
Definir: \
1- Ns = número de simulaciones \
2- Cholesky o CP. \
3- Distribución Normal o empírica. \
4- Nivel de toleracia en CP. \



In [0]:
%%R
Ns = 10000 
chcp = 1 #1 CP, 0 cholesky
normempy = 1 #0 Normal, 1 empírico
eta = 0.9 #nivel de varianza para CP 

if(chcp==0){
    mT = ncol(X_port)
    VarDX = cor(DeltaX_port)
    CvarDX = chol(VarDX,pivot=TRUE)
    MatrixSim = matrix(0,Ns,mT)
    if(normempy==0){
        for(i in 1:mT){
            MatrixSim[,i] = rnorm(Ns)*sd(DeltaX_port[,i])*(ncol(DeltaX_port)/(ncol(DeltaX_port)-1))+mean(DeltaX_port[,i]) 
        }
        Met = 'Combinación Cholesky Normal'
    }else{
        for(i in 1:mT){
            MatrixSim[,i] = quantile(DeltaX_port[,i],runif(Ns)) #empírica
        }
        Met = 'Combinación Cholesky Empírica'
    }
    dx_s = MatrixSim %*% CvarDX   #factores de riesgo simulados
}else{
    VarDX = cov(DeltaX_port)
    G =eigen(VarDX)
    eigval = G$values   #eigenvalores; la suma de todos los eigen valores es igual a la varianza
    eigvalp = eigval/sum(eigval)  #porcentajes, cuánto representa cada eigenvalor respecto al total de la varianza.
    eigvalcum = cumsum(eigvalp) #sumamos cada eigenvalor acumulativamente
    k = which.max( eigvalcum > eta )  #¿con cuántos eigen valores nos quedamos dado el valor eta?
    y=as.matrix((DeltaX_port-colMeans(DeltaX_port)))%*%G$vectors[,1:k] #cálculo de componentes principales    
    MatrixSim = matrix(0,Ns,k)
    if(normempy==0){
        for(i in 1:k){
            MatrixSim[,i] = rnorm(Ns)*sd(y[,i])*(ncol(y)/(ncol(y)-1))+mean(y[,i]) 
        }
        Met = 'Combinación CP Normal'
    }else{
        for(i in 1:k){
            MatrixSim[,i] = quantile(y[,i],runif(Ns)) #empírica
        }
        Met = 'Combinación CP Empírica'
    }
    dx_s = MatrixSim %*% t(G$vectors[,1:k])   #factores de riesgo simulados, matríz de Ns x m donde m son el número de factores de riesgo.
    #print(head(dx_s))
}

#print(matrix(1,1,ncol(dx_s))%*%var(dx_s)%*%t(matrix(1,1,ncol(dx_s)))/matrix(1,1,ncol(dx_s))%*%var(DeltaX_port)%*%t(matrix(1,1,ncol(dx_s))))
print(Met)
#print(sum(diag(var(dx_s)))/sum(diag(var(DeltaX_port))))


[1] "Combinación CP Empírica"


#Medición de Riesgo.

In [0]:
%%R
#Cálculo de matriz de pérdidas y ganancias Opciones Tasa de interés
#dimensión
m=ncol(plazos_oir)  #PASO CLAVE, total de instrumentos
X_s_oir=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en dx_s: x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m) #factores de riesgo del primer tipo considerando el total de instrumentos.
Vfr2_oir=matrix(0,Ns,m) #factores de riesgo del segundo tipo considerando el total de instrumentos.
Vfr3_oir=matrix(0,Ns,m) #factores de riesgo del tercer tipo considerando el total de instrumentos.
PG_oir=matrix(0,Ns,m) #Pérdidas y ganancias totales
PGfr1_oir=matrix(0,Ns,m) #Pérdidas y ganancias del primer factor de riesgo considerando el total de instrumentos.
PGfr2_oir=matrix(0,Ns,m)  #Pérdidas y ganancias del segundo factor de riesgo considerando el total de instrumentos.
PGfr3_oir=matrix(0,Ns,m)  #Pérdidas y ganancias del tercer factor de riesgo considerando el total de instrumentos.
PGT_oir=matrix(0,Ns,1)  #Pérdidas y ganancias totales.
PGfr1T_oir=matrix(0,Ns,1) #Pérdidas y ganancias totales del primer factor de riesgo.
PGfr2T_oir=matrix(0,Ns,1) #Pérdidas y ganancias totales del segundo factor de riesgo.
PGfr3T_oir=matrix(0,Ns,1) #Pérdidas y ganancias totales del tercer factor de riesgo.

DeltaX_s_oir=dx_s[,1:n_if]  #de la matríz dx_s con los factores de riesgo simulados, nos quedamos con las últimas columnas correspondientes en este caso al número de factores de riesgo por todos los instrumentos.
x0_oir=X_oir[1,] #PASO CLAVE es el dato inicial de los factores de riesgo reales de todos los instrumentos, con 5 instrumentos obtenemos 15 valores. (3 por cada instrumento)

for (i in 1:Ns){
   X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,]) 
  #PASO CLAVE
  V_oir[i,] =opctint(X_s_oir[i,(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  
  PG_oir[i,]=V_oir[i,]-V0_oir         #diferencia de la valuación factores de riesgo simulados - valuación original
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir   #cálculo de pérdidas y ganancias factor de riesgo 1
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir   #cálculo de pérdidas y ganancias factor de riesgo 2
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir   #cálculo de pérdidas y ganancias factor de riesgo 3

  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])   #cálculo de pérdidas y ganancias totales factor de riesgo 1
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])   #cálculo de pérdidas y ganancias totales factor de riesgo 2
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])   #cálculo de pérdidas y ganancias totales factor de riesgo 3
}

VaRCont_oir=matrix(0,1,m)   #VaR por portafolio
VaRfr1_oir=matrix(0,1,m)    #VaR factor de riesgo 1
VaRfr2_oir=matrix(0,1,m)    #VaR factor de riesgo 2
VaRfr3_oir=matrix(0,1,m)    #VaR factor de riesgo 3
CVaRCont_oir=matrix(0,1,m)  #CVaR por portafolio
CVaRfr1_oir=matrix(0,1,m)   #CVaR factor de riesgo 1
CVaRfr2_oir=matrix(0,1,m)   #CVaR factor de riesgo 2
CVaRfr3_oir=matrix(0,1,m)   #CVaR factor de riesgo 3
for (i in (1:m))
{
  VaRCont_oir[i]=quantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=quantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=quantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=quantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}
Met
VaRCont_oir
VaRfr1_oir
VaRfr2_oir
CVaRCont_oir
CVaRfr1_oir
CVaRfr2_oir


#VaR Total
Met
VaRTotal_oir=quantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=quantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=quantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=quantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


print(cbind(Valor_del_portafolio=V0T_port))
print('')
print(cbind(VaR_total=VaRTotal_oir, VaR_por_isntrumento=VaRCont_oir))#, V0_oir))
print('')
print(cbind(CVaR_total=CVaRTotal_oir, CVaR_por_isntrumento=CVaRCont_oir))
print('')
print(cbind(VaR_total=VaRTotal_oir,VaR_total_factorRiesgo1=VaRTotalfr1_oir,VaR_total_factorRiesgo2=VaRTotalfr2_oir,VaR_total_factorRiesgo3=VaRTotalfr3_oir))
print('')
print(cbind(CVaR_total=CVaRTotal_oir,CVaR_total_factorRiesgo1=CVaRTotalfr1_oir,CVaR_total_factorRiesgo2=CVaRTotalfr2_oir,CVaR_total_factorRiesgo3=CVaRTotalfr3_oir))


     Valor_del_portafolio
[1,]             2.242579
[1] ""
    VaR_total                                                         
5% -0.1872886 -0.04485874 -0.1187818 -0.1322619 -0.01733624 -0.1631379
[1] ""
     CVaR_total                                                      
[1,]  -0.286494 -0.0643751 -0.40983 -0.2027712 -0.05830492 -0.2530683
[1] ""
    VaR_total VaR_total_factorRiesgo1 VaR_total_factorRiesgo2
5% -0.1872886           -5.516589e-05            -0.004911658
   VaR_total_factorRiesgo3
5%              -0.1934031
[1] ""
     CVaR_total CVaR_total_factorRiesgo1 CVaR_total_factorRiesgo2
[1,]  -0.286494            -0.0001481229              -0.01310366
     CVaR_total_factorRiesgo3
[1,]                0.5768829


#Riesgo Integral.

In [0]:
%%R
#Medición de riesgo por factor de riesgo de todo el portafolios.

#Tasas de interés
PGPort_TI=PGfr1T_oir +PGfr2T_oir
VaRPort_TI=quantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Volatilidad
PGPort_VOL=PGfr3T_oir  #Pérdidas y ganancias
VaRPort_VOL=quantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR

PGT_Port=PGPort_TI+PGPort_VOL
VaRTotal_Port=quantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR
print(cbind(VaR_total_portafolio=VaRTotal_Port))
print(cbind(CVaR_total_portafolio=CVaRTotal_Port))
print(cbind(Valor_del_portafolio=V0T_port))

   VaR_total_portafolio
5%           -0.1871662
     CVaR_total_portafolio
[1,]            -0.2860572
     Valor_del_portafolio
[1,]             2.242579
